<a href="https://colab.research.google.com/github/andrewsongj/fraud-detector/blob/main/NetSec_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [4]:
! pip install -q kaggle

from google.colab import files

In [5]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jjg5fg","key":"e2702d4aa79e9df32c4bcd6b16b5fd12"}'}

In [6]:
! mkdir ~/.kaggle

In [7]:
! cp kaggle.json ~/.kaggle/

In [8]:
! chmod 600 ~/.kaggle/kaggle.json



In [11]:
!kaggle datasets download -d emilynack/aci-iot-network-traffic-dataset-2023

100% 32.3G/32.3G [20:55<00:00, 26.0MB/s]
100% 32.3G/32.3G [20:55<00:00, 27.6MB/s]


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
!mv aci-iot-network-traffic-dataset-2023.zip /content/drive/MyDrive/

In [16]:
import pandas as pd
!unzip /content/drive/MyDrive/aci-iot-network-traffic-dataset-2023.zip

Archive:  /content/drive/MyDrive/aci-iot-network-traffic-dataset-2023.zip
  inflating: ACI-IoT-2023-README.docx  
  inflating: ACI-IoT-2023-README.txt  
  inflating: ACI-IoT-2023.csv        
  inflating: Timesheets/Timesheets/ACI-IoT-2023 Data Collection Timesheet - Benign 1.csv  
  inflating: Timesheets/Timesheets/ACI-IoT-2023 Data Collection Timesheet - Benign 2.csv  
  inflating: Timesheets/Timesheets/ACI-IoT-2023 Data Collection Timesheet - Brute Force+Spoofing.csv  
  inflating: Timesheets/Timesheets/ACI-IoT-2023 Data Collection Timesheet - DoS.csv  
  inflating: Timesheets/Timesheets/ACI-IoT-2023 Data Collection Timesheet - Recon.csv  
  inflating: pcap_combined/Combined_Pcaps/Benign Pcaps/2023-10-31-08_59_06_wireless.pcap  
  inflating: pcap_combined/Combined_Pcaps/Benign Pcaps/2023-10-31-08_59_09_wired.pcap  
  inflating: pcap_combined/Combined_Pcaps/Benign Pcaps/2023-10-31-09_00_06_wireless.pcap  
  inflating: pcap_combined/Combined_Pcaps/Benign Pcaps/2023-10-31-09_00_09_wired

In [19]:
df = pd.read_csv("ACI-IoT-2023.csv")

In [21]:
df.drop(["Flow Bytes/s","Timestamp","Flow Packets/s"],axis=1,inplace=True)

In [34]:
selected_labels = ['ICMP Flood', 'Slowloris', 'SYN Flood', 'UDP Flood', 'DNS Flood', 'Benign']
df =df[df['Label'].isin(selected_labels)]

In [55]:
features = df.columns.tolist()
features.remove("Label")
features.remove("Flow ID")
features.remove("Src IP")
features.remove("Dst IP")
X = df[features]
y=df['Label']

In [57]:
X

,Src Port,Dst Port,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Connection Type
0,33344,443,6,379933,11,11,720.0,6169.0,517.0,0.0,...,32,0.000000e+00,0.000000e+00,0.0,0.0,1.698846e+15,0.000000e+00,1.698846e+15,1.698846e+15,wired
1,36754,80,6,205637,3,3,87.0,189.0,87.0,0.0,...,32,0.000000e+00,0.000000e+00,0.0,0.0,1.698846e+15,0.000000e+00,1.698846e+15,1.698846e+15,wired
2,36754,80,6,0,2,0,0.0,0.0,0.0,0.0,...,32,0.000000e+00,0.000000e+00,0.0,0.0,1.698846e+15,0.000000e+00,1.698846e+15,1.698846e+15,wired
3,8080,40426,6,5030379,1,2,6.0,0.0,6.0,6.0,...,20,0.000000e+00,0.000000e+00,0.0,0.0,8.494231e+14,1.201266e+15,1.698846e+15,5.030379e+06,wired
4,40054,80,6,72278,3,4,87.0,148.0,87.0,0.0,...,32,0.000000e+00,0.000000e+00,0.0,0.0,1.698846e+15,0.000000e+00,1.698846e+15,1.698846e+15,wired
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231406,58126,443,6,1,2,0,12.0,0.0,6.0,6.0,...,20,0.000000e+00,0.000000e+00,0.0,0.0,1.698946e+15,0.000000e+00,1.698946e+15,1.698946e+15,wired
1231407,30001,47579,6,50077229,4,2,140.0,390.0,70.0,0.0,...,32,7.708200e+04,0.000000e+00,77082.0,77082.0,8.494730e+14,1.201336e+15,1.698946e+15,4.992308e+07,wired
1231408,0,0,0,46365348,15,0,0.0,0.0,0.0,0.0,...,0,3.713166e+06,3.789721e+06,8045778.0,1014561.0,4.247365e+14,8.494730e+14,1.698946e+15,6.831749e+06,wired
1231409,52046,443,6,146262,15,37,20340.0,14242.0,9327.0,0.0,...,20,0.000000e+00,0.000000e+00,0.0,0.0,1.698946e+15,0.000000e+00,1.698946e+15,1.698946e+15,wired


In [58]:
# prompt: one hot encode connection field in X

import pandas as pd
X = pd.get_dummies(X, columns=["Connection Type"],dtype=int)


In [68]:
# prompt: prep data for keras

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0, stratify=y)
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_train_encoded = encoder.fit_transform(y_train)
y_test_encoded = encoder.transform(y_test)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [69]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Build the model
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(len(encoder.classes_), activation='softmax')  # Output layer
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 256)               20480     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_17 (Dense)            (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_18 (Dense)            (None, 64)                8256      
                                                                 
 dense_19 (Dense)            (None, 6)                 390       
                                                                 
Total params: 62022 (242.27 KB)
Trainable params: 62022 

In [70]:
history = model.fit(X_train_scaled, y_train_encoded, epochs=20, batch_size=32, validation_split=0.2)
test_loss, test_acc = model.evaluate(X_test_scaled, y_test_encoded)
print(f"Test Accuracy: {test_acc}")

Epoch 1/20
11902/11902 [==============================] - 46s 4ms/step - loss: 0.0380 - accuracy: 0.9889 - val_loss: 0.0134 - val_accuracy: 0.9966
Epoch 2/20
11902/11902 [==============================] - 42s 4ms/step - loss: 0.0192 - accuracy: 0.9950 - val_loss: 0.0125 - val_accuracy: 0.9964
Epoch 3/20
11902/11902 [==============================] - 42s 3ms/step - loss: 0.0167 - accuracy: 0.9956 - val_loss: 0.0111 - val_accuracy: 0.9968
Epoch 4/20
11902/11902 [==============================] - 42s 4ms/step - loss: 0.0166 - accuracy: 0.9955 - val_loss: 0.0086 - val_accuracy: 0.9975
Epoch 5/20
11902/11902 [==============================] - 41s 3ms/step - loss: 0.0142 - accuracy: 0.9963 - val_loss: 0.0076 - val_accuracy: 0.9978
Epoch 6/20
11902/11902 [==============================] - 42s 4ms/step - loss: 0.0149 - accuracy: 0.9963 - val_loss: 0.0080 - val_accuracy: 0.9977
Epoch 7/20
11902/11902 [==============================] - 41s 3ms/step - loss: 0.0155 - accuracy: 0.9960 - val_loss: 0

In [72]:
model.save('/content/drive/MyDrive/my_network_model.h5')  # Saves as a single HDF5 file